In [1]:
from pyflann import *
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from math import sqrt
import copy
import timeit
mydata = pd.read_csv('subset_numeric.csv')
#mydata = pd.read_csv('subset100k.csv')
#del mydata["orig_destination_distance"]
#dat = mydata.values

In [2]:
m = np.zeros((10000,100))
m[m==0]=-1
# Build the dataset Matrix
# -1: no click
# 1: click but no booking
# >1: number of bookings
userid = mydata['user_id'].unique()
hotelcluster = mydata['hotel_cluster'].unique()
for index, row in mydata.iterrows():
    rowNum = np.where(userid==row['user_id'])
    colNum = np.where(hotelcluster==row['hotel_cluster'])
    if m[rowNum,colNum]==-1:
        m[rowNum,colNum]=row['is_booking']+1
    else:
        m[rowNum,colNum] = m[rowNum,colNum]+row['is_booking']
print m.max()

37.0


In [6]:
def splitDataSet(percent,m):
    pMatrix = np.random.rand(10000,100)
    TrainMatrix = np.zeros([10000,100])
    TestMatrix = np.zeros([10000,100])
    for index, x in np.ndenumerate(pMatrix):
        if x >= percent:
            TrainMatrix[index] = m[index]
        else:
            TestMatrix[index] = m[index]
    TMatrix = copy.deepcopy(TrainMatrix)
    TMatrix[TMatrix<0]=0
    TTMatrix = copy.deepcopy(TestMatrix)
    TTMatrix[TTMatrix<0]=0
    return TrainMatrix,TestMatrix,TMatrix,TTMatrix

def predict(result, rowNum, colNum, TMatrix,user_similarity):
    topUser = result[rowNum]
    topSimilarity = user_similarity[rowNum][topUser]
    if topSimilarity.max()==0:
        prediction = 0.0
    else:
        prediction = (TMatrix[topUser][:,colNum].T.dot(topSimilarity))/sum(topSimilarity)
    return prediction

flann = FLANN()
def runANN(k,num_trees, TMatrix):
    result, dists = flann.nn(TMatrix, TMatrix, num_neighbors = k, algorithm = "kdtree", trees = num_trees)
    return result

def runKmeans(k,num_branch,iters,TMatrix):
    result, dists = flann.nn(TMatrix, TMatrix, num_neighbors = k, algorithm = "kmeans",
                             branching = num_branch, iterations = iters)
    return result

# (l,u): range of tuning variable, l needs to be greater than 2
def predict_all(l,u):
    rmse = []
    for i in range(l,u+1):
        rmse_i = []
        for j in range(1,6):
            TrainMatrix,TestMatrix,TMatrix,TTMatrix = splitDataSet(0.2,m)
            user_similarity = cosine_similarity(TMatrix)
            result = runANN(i,20,TMatrix)
            rMatrix = np.zeros([10000,100])
            for index, item in np.ndenumerate(TrainMatrix):
                if item==0:
                    rMatrix[index] = predict(result,index[0],index[1],TMatrix,user_similarity)
            rmseMatrix = TTMatrix-rMatrix
            rmse_i.append(sqrt(np.square(rmseMatrix).sum() / (1.0 * np.count_nonzero(TestMatrix))))
        print i, np.mean(rmse_i)
        rmse.append(np.mean(rmse_i))
    print rmse
    return rmse

def splitDataSet_2(m):
    pMatrix = np.random.rand(10000,100)
    Matrix_1 = np.zeros([10000,100])
    Matrix_2 = np.zeros([10000,100])
    Matrix_3 = np.zeros([10000,100])
    Matrix_4 = np.zeros([10000,100])
    Matrix_5 = np.zeros([10000,100])
    
    for index, x in np.ndenumerate(pMatrix):
        if x < 0.2:
            Matrix_1[index] = m[index]
        elif x>=0.2 and x<0.4:
            Matrix_2[index] = m[index]
        elif x>=0.4 and x<0.6:
            Matrix_3[index] = m[index]
        elif x>=0.6 and x<0.8:
            Matrix_4[index] = m[index]
        elif x>=0.8:
            Matrix_5[index] = m[index]
    return [Matrix_1,Matrix_2,Matrix_3,Matrix_4,Matrix_5]

def recommendation(k):
    Matrix = splitDataSet_2(m)
    resultMatrix = np.zeros([10000,100])
    for i in range(0,5):
        TestMatrix = copy.deepcopy(Matrix[i])
        TrainMatrix = np.zeros([10000,100])
        for j in range(0,5):
            if j != i:
                TrainMatrix += copy.deepcopy(Matrix[j])
        TMatrix = copy.deepcopy(TrainMatrix)
        TMatrix[TMatrix<0]=0
        TTMatrix = copy.deepcopy(TestMatrix)
        TTMatrix[TTMatrix<0]=0
        user_similarity = cosine_similarity(TMatrix)
        result = runANN(k,20,TMatrix)
        rMatrix = np.zeros([10000,100])
        for index, item in np.ndenumerate(TrainMatrix):
            if item==0:
                rMatrix[index] = predict(result,index[0],index[1],TMatrix,user_similarity)
        resultMatrix += copy.deepcopy(rMatrix)
    return resultMatrix

In [7]:
start_time = timeit.default_timer()
predict_all(38,38)
elapsed = timeit.default_timer()-start_time
print elapsed

38 0.425608828925
[0.42560882892547214]
49.1885972023


In [217]:
result = recommendation(38)

#Popular items in the resulted matrix
Rpopularity = np.sum(result, axis=0)
print np.argsort(Rpopularity)[-10:,]

#Popular items in the original matrix
Rpopularity = np.sum(m, axis=0)
print np.argsort(Rpopularity)[-10:,]

#Coverage of the recommendation based on predicted results
recommendation = np.zeros([10000,5])
for rowNum in range(result.shape[0]):
    recommend = np.argsort(result[rowNum])[-5:,]
    recommendation[rowNum]=recommend
print np.unique(recommendation)
print len(np.unique(recommendation))

#Coverage of the recomendation based on original dataset
Mrecommendation = np.zeros([10000,5])
for rowN in range(m.shape[0]):
    Mrecommend = np.argsort(m[rowN])[-5:,]
    Mrecommendation[rowN]=Mrecommend
print np.unique(Mrecommendation)
print len(np.unique(Mrecommendation))

#Count the number of same recommendations given by our result compared to the original matrix
count = 0
for user in range(result.shape[0]):
    recommend = set(np.argsort(result[user])[-5:,])
    Mrecomend = set(np.argsort(m[user])[-5:,])
    if (recommend.intersection(Mrecomend) != set([])):
        count+=1
print count

[20 22 77 58 80 43 62  1 70 49]
[77 37 80 58 43 20 62 70  1 49]
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.
  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.
  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.  71.  72.  73.  74.
  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.  85.  86.  87.  88.  89.
  90.  91.  92.  93.  94.  95.  96.  97.  98.  99.]
100
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.
  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.
  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.  71.  72.  73.  74.
  75.  76.  77.  78.  79.  80.  81.  82.  83